In [0]:
# Bài toán : cho một bộ dữ liệu dataset ngẫu nhiên từ file csv, hãy thực hiện các bước training model random forest sau để tính được chỉ số đánh giá hiệu quả mô hình ROC

In [0]:
#STEP 0 : Import và nhập dữ liệu 
import time
# Basic libs
import numpy as np
# from scipy.signal import find_peaks
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import builtins

# Sklearn 
import sklearn
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, confusion_matrix, precision_recall_curve

#Vector libs
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.feature import VectorAssembler, VectorSlicer
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MinMaxScaler  

# model libs
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.recommendation import ALS

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.clustering import KMeans
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.ml.classification import LogisticRegression

# Evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

#From spark
from pyspark.ml.stat import Correlation


# SQL function libs
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import col, asc



spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()


# df2 = spark.read.option("header",True) \
#      .csv("dbfs:/FileStore/model_test_csv.csv")
# df2.printSchema()
   
# df3 = spark.read.options(header='True', delimiter=',') \
#   .csv("/tmp/resources/zipcodes.csv")
# df3.printSchema()

schema = StructType() \
      .add("isdn",StringType(),True) \
      .add("fe1",DoubleType(),True) \
      .add("fe2",DoubleType(),True) \
      .add("fe3",DoubleType(),True) \
      .add("fe4",DoubleType(),True) \
      .add("fe5",DoubleType(),True) \
      .add("fe6",DoubleType(),True) \
      .add("fe7",DoubleType(),True) \
      .add("fe8",DoubleType(),True) \
      .add("fe9",DoubleType(),True) \
      .add("fe10",DoubleType(),True) \
      .add("fe11",DoubleType(),True) \
      .add("fe12",DoubleType(),True) \
      .add("fe13",DoubleType(),True) \
      .add("fe14",DoubleType(),True) \
      .add("fe15",DoubleType(),True) \
      .add("fe16",DoubleType(),True) \
      .add("fe17",DoubleType(),True) \
      .add("fe18",DoubleType(),True) \
      .add("fe19",DoubleType(),True) \
      .add("fe20",DoubleType(),True) \
      .add("fe21",DoubleType(),True) \
      .add("fe22",DoubleType(),True) \
      .add("fe23",DoubleType(),True) \
      .add("fe24",DoubleType(),True) \
      .add("fe25",DoubleType(),True) \
      .add("fe26",DoubleType(),True) \
      .add("fe27",DoubleType(),True) \
      .add("fe28",DoubleType(),True) \
      .add("fe29",DoubleType(),True) \
      .add("fe30",DoubleType(),True) \
      .add("label",IntegerType(),True) 
      
df_with_schema = spark.read.format("csv") \
    .option("header", True) \
    .schema(schema)\
    .load("dbfs:/mllib/model_exam_csv_1.csv")
#     .csv("/FileStore/model_exam_csv_1.csv")
#     .csv("dbfs:/FileStore/model_exam_csv_1.csv")

df_with_schema.printSchema()



root
 |-- isdn: string (nullable = true)
 |-- fe1: double (nullable = true)
 |-- fe2: double (nullable = true)
 |-- fe3: double (nullable = true)
 |-- fe4: double (nullable = true)
 |-- fe5: double (nullable = true)
 |-- fe6: double (nullable = true)
 |-- fe7: double (nullable = true)
 |-- fe8: double (nullable = true)
 |-- fe9: double (nullable = true)
 |-- fe10: double (nullable = true)
 |-- fe11: double (nullable = true)
 |-- fe12: double (nullable = true)
 |-- fe13: double (nullable = true)
 |-- fe14: double (nullable = true)
 |-- fe15: double (nullable = true)
 |-- fe16: double (nullable = true)
 |-- fe17: double (nullable = true)
 |-- fe18: double (nullable = true)
 |-- fe19: double (nullable = true)
 |-- fe20: double (nullable = true)
 |-- fe21: double (nullable = true)
 |-- fe22: double (nullable = true)
 |-- fe23: double (nullable = true)
 |-- fe24: double (nullable = true)
 |-- fe25: double (nullable = true)
 |-- fe26: double (nullable = true)
 |-- fe27: double (nullable = tr

In [0]:
display(df_with_schema)

# df_with_schema.show()

isdn,fe1,fe2,fe3,fe4,fe5,fe6,fe7,fe8,fe9,fe10,fe11,fe12,fe13,fe14,fe15,fe16,fe17,fe18,fe19,fe20,fe21,fe22,fe23,fe24,fe25,fe26,fe27,fe28,fe29,fe30,label
a1,21.0,7.0,5.0,1.0,25.0,49.0,1.0,78.0,97.0,42.0,71.0,53.0,1.0,68.0,23.0,88.0,100.0,98.0,1.0,65.0,30.0,31.0,100.0,13.0,78.0,18.0,10.0,10.0,96.0,80.0,64
a2,94.0,33.0,5.0,2.0,66.0,11.0,1.0,75.0,24.0,28.0,6.0,30.0,2.0,40.0,88.0,36.0,32.0,33.0,1.0,12.0,1.0,33.0,80.0,85.0,77.0,63.0,10.0,14.0,50.0,100.0,57
a3,51.0,92.0,5.0,3.0,25.0,99.0,1.0,79.0,44.0,22.0,92.0,20.0,3.0,1.0,100.0,61.0,39.0,53.0,1.0,53.0,18.0,51.0,97.0,61.0,37.0,12.0,10.0,46.0,65.0,94.0,89
a4,16.0,25.0,5.0,4.0,1.0,4.0,1.0,70.0,65.0,91.0,73.0,88.0,4.0,58.0,75.0,39.0,68.0,35.0,1.0,9.0,66.0,52.0,96.0,28.0,42.0,26.0,10.0,70.0,100.0,85.0,35
a5,80.0,4.0,5.0,5.0,77.0,24.0,1.0,97.0,58.0,44.0,84.0,75.0,5.0,90.0,85.0,30.0,9.0,37.0,1.0,60.0,3.0,1.0,84.0,55.0,56.0,97.0,10.0,21.0,92.0,89.0,58
a6,82.0,66.0,5.0,6.0,2.0,25.0,1.0,72.0,56.0,32.0,72.0,75.0,6.0,22.0,11.0,66.0,49.0,39.0,1.0,31.0,55.0,83.0,90.0,69.0,42.0,93.0,10.0,69.0,56.0,3.0,85
a7,42.0,91.0,5.0,7.0,33.0,14.0,1.0,100.0,28.0,61.0,4.0,77.0,7.0,93.0,46.0,23.0,89.0,58.0,1.0,95.0,91.0,61.0,93.0,73.0,33.0,81.0,10.0,82.0,22.0,32.0,62
a8,43.0,68.0,5.0,8.0,24.0,75.0,1.0,47.0,64.0,15.0,75.0,99.0,8.0,67.0,82.0,2.0,14.0,34.0,1.0,45.0,56.0,99.0,88.0,69.0,1.0,68.0,10.0,27.0,70.0,27.0,73
a9,11.0,72.0,5.0,9.0,28.0,70.0,1.0,40.0,11.0,53.0,26.0,48.0,9.0,4.0,27.0,61.0,24.0,49.0,1.0,9.0,96.0,57.0,93.0,1.0,98.0,17.0,10.0,65.0,87.0,100.0,29
a10,41.0,79.0,5.0,10.0,22.0,73.0,1.0,68.0,38.0,18.0,14.0,47.0,10.0,38.0,89.0,25.0,8.0,39.0,1.0,27.0,54.0,59.0,88.0,15.0,100.0,93.0,10.0,41.0,58.0,49.0,71


In [0]:
#STEP 1 : Tính tỷ lệ phần tử label 0 , label 1,

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-2048736610454351>:2
      1 # rc = spark.read.csv('/path/file.csv', header=True)
----> 2 df_with_schema.write.format("parquet").save('D:/work/3.Document_knowledge')

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1398, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
   1396     self._jwrite.save()
   1397 else:
-> 1398     self._jwrite.save(path)

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py

In [0]:
df_with_schema.show()
df_with_schema.count()

+----+----+----+---+----+----+-----+---+-----+----+----+----+----+----+----+-----+----+-----+----+----+----+----+----+-----+----+-----+----+----+-----+-----+-----+-----+
|isdn| fe1| fe2|fe3| fe4| fe5|  fe6|fe7|  fe8| fe9|fe10|fe11|fe12|fe13|fe14| fe15|fe16| fe17|fe18|fe19|fe20|fe21|fe22| fe23|fe24| fe25|fe26|fe27| fe28| fe29| fe30|label|
+----+----+----+---+----+----+-----+---+-----+----+----+----+----+----+----+-----+----+-----+----+----+----+----+----+-----+----+-----+----+----+-----+-----+-----+-----+
|  a1|21.0| 7.0|5.0| 1.0|25.0| 49.0|1.0| 78.0|97.0|42.0|71.0|53.0| 1.0|68.0| 23.0|88.0|100.0|98.0| 1.0|65.0|30.0|31.0|100.0|13.0| 78.0|18.0|10.0| 10.0| 96.0| 80.0|   64|
|  a2|94.0|33.0|5.0| 2.0|66.0| 11.0|1.0| 75.0|24.0|28.0| 6.0|30.0| 2.0|40.0| 88.0|36.0| 32.0|33.0| 1.0|12.0| 1.0|33.0| 80.0|85.0| 77.0|63.0|10.0| 14.0| 50.0|100.0|   57|
|  a3|51.0|92.0|5.0| 3.0|25.0| 99.0|1.0| 79.0|44.0|22.0|92.0|20.0| 3.0| 1.0|100.0|61.0| 39.0|53.0| 1.0|53.0|18.0|51.0| 97.0|61.0| 37.0|12.0|10.0| 46.0

In [0]:
df_with_schema.select('isdn').show()

+----+
|isdn|
+----+
|  a1|
|  a2|
|  a3|
|  a4|
|  a5|
|  a6|
|  a7|
|  a8|
|  a9|
| a10|
| a11|
| a12|
| a13|
| a14|
| a15|
| a16|
| a17|
| a18|
| a19|
| a20|
+----+
only showing top 20 rows



In [0]:
# BƯỚC 2: LỌC VÀ CHỈ LẤY CỘT FEATURES TỪ fe1 -> fe30 
print( len(df_with_schema.columns))
print(df_with_schema.columns)
# number_cols_m =  train_m.columns 
cols = df_with_schema.columns


number_cols = [ x for x in cols if x not in [ 'isdn', 'label'] ]

print(number_cols)




32
['isdn', 'fe1', 'fe2', 'fe3', 'fe4', 'fe5', 'fe6', 'fe7', 'fe8', 'fe9', 'fe10', 'fe11', 'fe12', 'fe13', 'fe14', 'fe15', 'fe16', 'fe17', 'fe18', 'fe19', 'fe20', 'fe21', 'fe22', 'fe23', 'fe24', 'fe25', 'fe26', 'fe27', 'fe28', 'fe29', 'fe30', 'label']
['fe1', 'fe2', 'fe3', 'fe4', 'fe5', 'fe6', 'fe7', 'fe8', 'fe9', 'fe10', 'fe11', 'fe12', 'fe13', 'fe14', 'fe15', 'fe16', 'fe17', 'fe18', 'fe19', 'fe20', 'fe21', 'fe22', 'fe23', 'fe24', 'fe25', 'fe26', 'fe27', 'fe28', 'fe29', 'fe30']


In [0]:
# BƯỚC 3: TẠO VECTOR
class_col = 'label'
class_name = ['model_test']
primary_keys = [ 'isdn']

#

vector_feature_columns = number_cols

assembler = VectorAssembler(inputCols = vector_feature_columns , outputCol='features_m')

# m
train_df_transformed = assembler.transform(df_with_schema)  # chạy lệnh tạo vecto

print(train_df_transformed)       



DataFrame[isdn: string, fe1: double, fe2: double, fe3: double, fe4: double, fe5: double, fe6: double, fe7: double, fe8: double, fe9: double, fe10: double, fe11: double, fe12: double, fe13: double, fe14: double, fe15: double, fe16: double, fe17: double, fe18: double, fe19: double, fe20: double, fe21: double, fe22: double, fe23: double, fe24: double, fe25: double, fe26: double, fe27: double, fe28: double, fe29: double, fe30: double, label: int, features_m: vector]


In [0]:
# BƯỚC 4 TẠO BẢNG ĐẦU VÀO MODEL 
train_m = train_df_transformed.select('isdn', 'features_m', class_col)   
train_m.columns
# print( type(train_m))

Out[97]: ['isdn', 'features_m', 'label']

In [0]:
#BƯỚC 5: TRUYỀN THAM SỐ CHO MODEL 
rf = RandomForestClassifier(  numTrees = 15,         
                                maxDepth=30,
                                labelCol=class_col,
                                featuresCol='features_m',
                                predictionCol='pred_{0}'.format(class_name[0]),
                                probabilityCol='prob_{0}'.format(class_name[0]),
                                rawPredictionCol='raw_pred_{0}'.format(class_name[0]),
                                minInstancesPerNode= 100 ,#argument_df[0][0],
                                maxBins=32,  
                                featureSubsetStrategy="auto",
                                #maxMemoryInMB = 256,
                                #minInfoGain=0.005,
                                subsamplingRate= 1,  
                                #cacheNodeIds=True, 
                                impurity='entropy',
                                #handleInvalid = c("error", "keep", "skip"),
                                # checkpointInterval = 10,
                                seed= 2022)    

In [0]:
# BƯỚC 6: CHẠY MODEL 
from pyspark.ml.evaluation import BinaryClassificationEvaluator
model_m = rf.fit(train_m)

In [0]:
# BƯỚC 7: ĐÁNH GIÁ MODEL BẰNG ROC 
evaluator = BinaryClassificationEvaluator(rawPredictionCol='raw_pred_{0}'.format(class_name[0]), labelCol=class_col, metricName="areaUnderROC")
train_roc_m = evaluator.evaluate(train_m)  
print('ROC_train: {0}'.format(train_roc_m))

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
<command-250342759212221> in <module>
      1 # BƯỚC 7: ĐÁNH GIÁ MODEL BẰNG ROC
      2 evaluator = BinaryClassificationEvaluator(rawPredictionCol='raw_pred_{0}'.format(class_name[0]), labelCol=class_col, metricName="areaUnderROC")
----> 3 train_roc_m = evaluator.evaluate(train_m)
      4 print('ROC_train: {0}'.format(train_roc_m))

/databricks/spark/python/pyspark/ml/evaluation.py in evaluate(self, dataset, params)
     82                 return self.copy(params)._evaluate(dataset)
     83             else:
---> 84                 return self._evaluate(dataset)
     85         else:
     86             raise TypeError("Params must be a param map but got %s." % type(params))

/databricks/spark/python/pyspark/ml/evaluation.py in _evaluate(self, dataset)
    118         """
    119         self._transfer_params_to_java()
--> 120         r